In [1]:
import pandas as pd
import gzip
import pickle
import json
import os

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from glob import glob 


def load_data(filedata):

    dataframe = pd.read_csv(
        filedata,
        index_col=False,
        compression="zip",
    )

    return dataframe

In [2]:
############## Paso 1 ##############
#Limpiar los datos
def clean_data(df):
    df_copy = df.copy()
    df_copy = df_copy.rename(columns={'default payment next month' : "default"})
    df_copy = df_copy.drop(columns=["ID"])
    df_copy = df_copy.loc[df["MARRIAGE"] != 0]
    df_copy = df_copy.loc[df["EDUCATION"] != 0]
    df_copy["EDUCATION"] = df_copy["EDUCATION"].apply(lambda x: 4 if x >= 4 else x)
    df_copy = df_copy.dropna()
    return df_copy


In [3]:
############## Paso 2 ##############
# Divida los datasets en x_train, y_train, x_test, y_test.
def split_data(df):
    #X , Y
    return df.drop(columns=["default"]), df["default"]

In [4]:
############## Paso 3 ##############
# Crear un pipeline para el modelo de clasificación
def make_pipeline(x_train):
    categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]
    numerical_features = [col for col in x_train.columns if col not in categorical_features]

    # Preprocesador
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_features),
            ('scaler', StandardScaler(), numerical_features),
        ]
    )


    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ('feature_selection', SelectKBest(score_func=f_classif)),  
        ('pca', PCA()),
        ('classifier', MLPClassifier(max_iter=15000,random_state=21))
    ])

    return pipeline


In [5]:
############## Paso 4 ##############
# Optimizar los hiperparametros del pipeline usando validación cruzada.
def create_estimator(pipeline):
    # Definición de la malla de parámetros para la búsqueda
    param_grid = {
        "pca__n_components": [None],
        "feature_selection__k": [20],
        "classifier__hidden_layer_sizes": [(50, 30, 40, 60)],
        "classifier__alpha": [0.26],
        'classifier__learning_rate_init': [0.001],
    }

    # Configuración de GridSearchCV para la validación cruzada
    grid_search = GridSearchCV(
        estimator=pipeline,           # Pipeline que incluye el preprocesamiento y el clasificador
        param_grid=param_grid,        # Hiperparámetros a optimizar
        cv=10,                        # 10 divisiones para la validación cruzada
        scoring='balanced_accuracy',
        n_jobs=-1,
        refit=True 
    )

    return grid_search

In [6]:
############## Paso 5 ##############
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
def _create_output_directory(output_directory):
    if os.path.exists(output_directory):
        for file in glob(f"{output_directory}/*"):
            os.remove(file)
        os.rmdir(output_directory)
    os.makedirs(output_directory)

def _save_model(path, estimator):
    _create_output_directory("files/models/")  # Verifica que el directorio se gestione correctamente.

    with gzip.open(path, "wb") as f:  # Abre el archivo comprimido en modo escritura binaria.
        pickle.dump(estimator, f)  # Guarda el modelo serializado.

In [7]:
############## Paso 6 ##############
# Calcule las metricas de precision, precision balanceada, recall...
def calculate_metrics(dataset_type, y_true, y_pred):
    """Calculate metrics"""
    return {
        "type": "metrics",
        "dataset": dataset_type,
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0),
    }

In [8]:
############## Paso 7 ##############
# Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba
def calculate_confusion(dataset_type, y_true, y_pred):
    """Confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    return {
        "type": "cm_matrix",
        "dataset": dataset_type,
        "true_0": {"predicted_0": int(cm[0][0]), "predicted_1": int(cm[0][1])},
        "true_1": {"predicted_0": int(cm[1][0]), "predicted_1": int(cm[1][1])},
    }

def _run_jobs():
    
    data_train= load_data("./files/input/train_data.csv.zip")
    data_test = load_data("./files/input/test_data.csv.zip")
    data_train = clean_data(data_train)
    data_test = clean_data(data_test)
    x_train, y_train = split_data(data_train)
    x_test, y_test = split_data(data_test)
    pipeline = make_pipeline(x_train)

    estimator = create_estimator(pipeline)
    estimator.fit(x_train, y_train)

    _save_model(
        os.path.join("files/models/", "model.pkl.gz"),
        estimator,
    )

    y_test_pred = estimator.predict(x_test)
    test_precision_metrics = calculate_metrics("test", y_test, y_test_pred)
    y_train_pred = estimator.predict(x_train)
    train_precision_metrics = calculate_metrics("train", y_train, y_train_pred)

    test_confusion_metrics = calculate_confusion("test", y_test, y_test_pred)
    train_confusion_metrics = calculate_confusion("train", y_train, y_train_pred)

    os.makedirs("files/output/", exist_ok=True)

    with open("files/output/metrics.json", "w", encoding="utf-8") as file:
        file.write(json.dumps(train_precision_metrics) + "\n")
        file.write(json.dumps(test_precision_metrics) + "\n")
        file.write(json.dumps(train_confusion_metrics) + "\n")
        file.write(json.dumps(test_confusion_metrics) + "\n")

In [9]:
if __name__ == "__main__":
    _run_jobs()